# Introduction

The COMPAS dataset consists of the results of a commercial algorithm called COMPAS (Correctional Offender Management Profiling for Alternative Sanctions), used to assess a convicted criminal’s likelihood of reoffending. COMPAS has been used by judges and parole officers and is widely known for its bias against African-Americans.

It is commonly used for fairness tasks and race is usually set as the sensitive attribute

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
import fairness_functions as fp


/Users/dani/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Load data and drop target & senstive attributes

#### Data Preprocessing
In this section, we prepare the dataset for model training by:
- Handling missing values.
- Identifying **race** as the sensitive attribute for fairness evaluation.
- Encoding categorical variables and dropping irrelevant features.
- Splitting the dataset into training and testing sets.


In [2]:
# Load the COMPAS dataset
df = pd.read_csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")

# Define target and sensitive attribute
target = "two_year_recid"
sensitive_col = "race"

# Drop rows missing the target or sensitive attribute
df = df.dropna(subset=[target, sensitive_col])

# Separate features and target
X = df.drop(columns=[target])
y = df[target]


### Drop irrelevant columns for prediction

In [3]:

# Explicitly drop columns that are not useful (e.g., names, IDs, dates, custody indicators)
columns_to_drop = ['name', 'first', 'last', 'id', 'compas_screening_date', 
                   'v_screening_date', 'in_custody', 'out_custody']


print("Dropping columns:", columns_to_drop)
X = X.drop(columns=columns_to_drop, errors='ignore')

Dropping columns: ['name', 'first', 'last', 'id', 'compas_screening_date', 'v_screening_date', 'in_custody', 'out_custody']


### Feature Selection
Certain columns, such as **names, IDs, dates, and custody indicators**, are not relevant for prediction and might introduce noise.
These columns have been removed to improve model performance and fairness analysis.


### Impute Nan Values

Imputes numeric Nan values with column mean and Nans in categorical columns with column mode

In [4]:

# Specify which columns are categorical based on domain knowledge.
categorical_cols = ['race','sex', 'age_cat', 'c_charge_degree', 'r_charge_degree', 
                    'vr_charge_degree', 'type_of_assessment', 'score_text', 
                    'v_type_of_assessment', 'v_score_text']

# All remaining columns will be considered numeric if not in our categorical list.
numeric_cols = [col for col in X.columns if col not in categorical_cols]

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

# Convert numeric columns to numeric dtype (forcing non-numeric values to NaN)
X_numeric = X[numeric_cols].apply(lambda col: pd.to_numeric(col, errors='coerce'))

# Fill missing values in numeric columns with the mean of each column.
X_numeric = X_numeric.fillna(X_numeric.mean())

# For categorical columns, first filter them: drop any with high cardinality.
max_unique_threshold = 20
filtered_categorical_cols = [col for col in categorical_cols if X[col].nunique() <= max_unique_threshold]
print("Filtered Categorical columns (<=20 unique values):", filtered_categorical_cols)

# Process the categorical columns: fill missing values with the mode.
X_categorical = X[filtered_categorical_cols].copy()
for col in filtered_categorical_cols:
    X_categorical[col] = X_categorical[col].fillna(X_categorical[col].mode()[0])

Numeric columns: ['dob', 'age', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas', 'c_charge_desc', 'is_recid', 'r_case_number', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number', 'vr_offense_date', 'vr_charge_desc', 'decile_score.1', 'screening_date', 'v_decile_score', 'priors_count.1', 'start', 'end', 'event']
Categorical columns: ['race', 'sex', 'age_cat', 'c_charge_degree', 'r_charge_degree', 'vr_charge_degree', 'type_of_assessment', 'score_text', 'v_type_of_assessment', 'v_score_text']
Filtered Categorical columns (<=20 unique values): ['race', 'sex', 'age_cat', 'c_charge_degree', 'r_charge_degree', 'vr_charge_degree', 'type_of_assessment', 'score_text', 'v_type_of_assessment', 'v_score_text']


### One-hot encode categorical features

In [5]:

# One-hot encode the filtered categorical columns using pandas' get_dummies, dropping the first category.
X_categorical_encoded = pd.get_dummies(X_categorical, drop_first=True)

# Combine numeric and one-hot encoded categorical columns.
X_processed = pd.concat([X_numeric, X_categorical_encoded], axis=1)

# Fill any remaining NaN values with 0.
X_processed = X_processed.fillna(0)

# Preserve the sensitive attribute for fairness evaluation.
sens = df[sensitive_col]

print("Shape of processed features:", X_processed.shape)


Shape of processed features: (7214, 64)


### Handling Missing Values & Encoding
- Missing numeric values are imputed with their **mean**.
- Missing categorical values are filled with the **mode**.
- One-hot encoding is applied to categorical features to prepare the data for machine learning models.


### Split data to train & test sets

In [6]:
# Split data and also split the sensitive attribute for evaluation
X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X_processed, y, sens, test_size=0.3, random_state=42
)


print("X train shape: ",X_train.shape)
print("X test shape: ",X_test.shape)

X train shape:  (5049, 64)
X test shape:  (2165, 64)


### Train and evaluate baseline model

#### Baseline Model - Logistic Regression
We start by training a **logistic regression model** without any fairness constraints.
This model serves as a benchmark to evaluate the impact of bias mitigation techniques.

The model is evaluated based on:
- **Accuracy**: Overall prediction correctness.
- **F1 Score**: Balance between precision and recall.
- **Demographic Parity Difference**: Measures whether different groups receive similar positive predictions.
- **Equalized Odds Difference**: Checks whether the model's errors are fairly distributed across groups.


In [7]:
# Train the logistic regression model
lr = LogisticRegression(random_state=42, max_iter=10000)
lr.fit(X_train, y_train)

# Predict on the test set with the baseline model
y_pred_baseline = lr.predict(X_test)

# Evaluate baseline performance metrics
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
f1_score_baseline = f1_score(y_test, y_pred_baseline)

# Evaluate fairness metrics for the baseline model
baseline_dp_diff = demographic_parity_difference(y_test, y_pred_baseline, sensitive_features=sens_test)
baseline_eo_diff = equalized_odds_difference(y_test, y_pred_baseline, sensitive_features=sens_test)

print("=== Baseline Model Metrics ===")
print("Accuracy:", baseline_accuracy)
print("F1 score:",f1_score_baseline) 
print("Demographic Parity Difference:", baseline_dp_diff)
print("Equalized Odds Difference:", baseline_eo_diff)


=== Baseline Model Metrics ===
Accuracy: 0.9866050808314087
F1 score: 0.9849818746763335
Demographic Parity Difference: 0.3603117505995205
Equalized Odds Difference: 0.020676691729323307


### Baseline Model Evaluation
The results indicate that while the **baseline model achieves high accuracy**, it also shows significant fairness disparities:
- The **Demographic Parity Difference** is high, meaning one group receives more positive predictions.
- The **Equalized Odds Difference** suggests that error rates are not evenly distributed.

To mitigate these biases, we will explore fairness-aware techniques.


### Naive Fairness Approach - Removing Sensitive Attributes
A simple approach to fairness is **removing the sensitive attribute (`race`)** from the dataset.
However, bias can still persist in features that are correlated with race.
This method will be compared to more sophisticated fairness-aware techniques.

In [8]:
# Process X_processed as before
# Drop sensitive columns from the entire processed dataset
sensitive_encoded_cols = [col for col in X_processed.columns if col.startswith(sensitive_col + '_')]
X_processed_no_sensitive = X_processed.drop(columns=sensitive_encoded_cols)

# Split the data
X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X_processed_no_sensitive, y, sens, test_size=0.3, random_state=42
)

# Train the logistic regression model
lr = LogisticRegression(random_state=42,max_iter=10000)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred_naive = lr.predict(X_test)

# Evaluate baseline performance metrics
naive_accuracy = accuracy_score(y_test, y_pred_naive)
f1_score_naive = f1_score(y_test, y_pred_naive)

# Evaluate fairness metrics for the baseline model
naive_dp_diff = demographic_parity_difference(y_test, y_pred_naive, sensitive_features=sens_test)
naive_eo_diff = equalized_odds_difference(y_test, y_pred_naive, sensitive_features=sens_test)

print("=== Naive Model Metrics ===")
print("Accuracy:", naive_accuracy)
print("F1 score:",f1_score_naive) 
print("Demographic Parity Difference:", naive_dp_diff)
print("Equalized Odds Difference:", naive_eo_diff)


=== Naive Model Metrics ===
Accuracy: 0.9861431870669746
F1 score: 0.9844559585492227
Demographic Parity Difference: 0.36121103117506004
Equalized Odds Difference: 0.022556390977443608


### Naive Model Evaluation
The results show that removing the sensitive attribute **does not eliminate bias completely**.
- Although fairness metrics improve slightly, disparities remain.
- More advanced bias mitigation strategies are required to better balance fairness and accuracy.


### Fairness-Aware Learning
We now experiment with fairness-aware machine learning techniques to **reduce bias while maintaining predictive performance**.
We test:
1. **Pre-processing**: Adjusting feature distributions to reduce bias before training.
2. **In-processing**: Training with fairness constraints to ensure bias-aware optimization.
3. **Post-processing**: Adjusting predictions after training to meet fairness criteria.

Each method will be evaluated on its trade-off between accuracy and fairness.

In [9]:
# Define candidate methods for each stage.
pre_methods = {
    "None": fp.pre_none,
    "Correlation_Remover": fp.pre_correlation_remover,
    "Sensitive_Resampling": fp.pre_sensitive_resampling  # new candidate
}

in_methods = {
    "Baseline": fp.in_baseline,
    "Reweighting": fp.in_reweighting,
    "Exponential_Gradient_Demogrphic_Parity": fp.in_expgrad_dp,
    "Exponential_Gradient_Equalized_Odds": fp.in_expgrad_eo
}

post_methods = {
    "None": fp.post_none,
    "Threshold_Demogrphic_Parity": fp.post_threshold_dp,
    "Threshold_Equalized_Odds": fp.post_threshold_eo
}

# Run experiments:
results = fp.run_experiments(pre_methods, in_methods, post_methods,
                             X_train, y_train, sens_train,
                             X_test, y_test, sens_test)


/Users/dani/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/dani/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

### Select only pareto optimal methods

In [10]:

objectives = {"f1_score": True,"accuracy":True, "Demographic_parity": False, "Equalized_odds": False}

frontier = fp.pareto_frontier(results, objectives)

print("Pareto Frontier configurations:")
for config, metrics in frontier.items():
    print(f"{config}: {metrics}")

Pareto Frontier configurations:
Pre-processing: None. In-training: Baseline. Post-processing:None: {'accuracy': 0.9879907621247113, 'f1_score': 0.9865563598759048, 'Demographic_parity': 0.36300959232613916, 'Equalized_odds': 0.020676691729323307}
Pre-processing: Correlation_Remover. In-training: Baseline. Post-processing:None: {'accuracy': 0.9861431870669746, 'f1_score': 0.984472049689441, 'Demographic_parity': 0.3621103117505996, 'Equalized_odds': 0.022556390977443608}
Pre-processing: Correlation_Remover. In-training: Baseline. Post-processing:Threshold_Equalized_Odds: {'accuracy': 0.9861431870669746, 'f1_score': 0.984472049689441, 'Demographic_parity': 0.36121103117506004, 'Equalized_odds': 0.024}
Pre-processing: Correlation_Remover. In-training: Reweighting. Post-processing:None: {'accuracy': 0.9861431870669746, 'f1_score': 0.9844074844074844, 'Demographic_parity': 0.3594124700239808, 'Equalized_odds': 0.023809523809523836}
Pre-processing: Correlation_Remover. In-training: Reweighti

### Results & Discussion
After testing multiple fairness-aware models, we identify the **Pareto-optimal** solutions that best balance fairness and accuracy.

Key observations:
- Some methods significantly **reduce demographic parity differences** but decrease accuracy.
- Other methods maintain accuracy while **improving fairness to some extent**.
- The best choice depends on the specific application and the acceptable trade-off between accuracy and fairness.


### Apply thresholds on biase and portion of retained accuracy

### Set thresholds on accurcy, demographic parity and equalized odds

In [11]:
f1_threshold = 0.90
accuracy_threshold = 0.90
demographic_parity_threshold = 0.25
equalized_odds_threshold = 0.25

In [12]:
# Filter results based on thresholds.
filtered = fp.filter_results(frontier, f1_threshold=f1_threshold,accuracy_threshold=accuracy_threshold,
                            dp_threshold=demographic_parity_threshold, eo_threshold=equalized_odds_threshold)

print("\nFiltered Results (satisfying thresholds):")
for config, metrics in filtered.items():
    print(config, metrics)


Filtered Results (satisfying thresholds):
Pre-processing: Correlation_Remover. In-training: Reweighting. Post-processing:Threshold_Demogrphic_Parity {'accuracy': 0.9214780600461894, 'f1_score': 0.9089935760171306, 'Demographic_parity': 0.09202637889688253, 'Equalized_odds': 0.2}
Pre-processing: Sensitive_Resampling. In-training: Reweighting. Post-processing:Threshold_Equalized_Odds {'accuracy': 0.9852193995381062, 'f1_score': 0.9835051546391752, 'Demographic_parity': 0.20180740653378942, 'Equalized_odds': 0.2}


### Conclusion
- The **baseline model** performs well in accuracy but exhibits significant fairness disparities.
- **Removing the sensitive attribute** is insufficient for mitigating bias.
- **Fairness-aware methods** provide better fairness-accuracy trade-offs.
- The **optimal model** depends on how much fairness is prioritized over predictive performance.

This analysis highlights the importance of incorporating fairness constraints in machine learning models used for criminal justice and similar applications.

